In [ ]:
import numpy as np
from netCDF4 import Dataset
import subprocess
from pathlib import Path
import histdef as histld

def solar_noon(leap,jday,lon):
    hour=12.0
    #calculate fraction of year
    foy=2.0*np.pi*(jday-1.+(hour-12.)/24.)/(365+leap)
    #equation of time (in minutes)
    eqtime=229.18*(0.000075+0.001868*np.cos(foy)-0.032077*np.sin(foy) \
                   -0.014615*np.cos(2.0*foy)-0.040849*np.sin(2.0*foy))
    #return solar in hour
    snoon=(720.-4.*lon-eqtime)/60.
    return snoon
#the following code extracts hourly weather forcing from the NLDAS 2 product.

lonc=-111.7620
latc=35.0890
datadir='/Volumes/toshiba/'
workdir='./'
years=[2021,2022]
daz=[31,28,31,30,31,30,31,31,30,31,30,31]


fnm='/Volumes/toshiba/NLDAS_FORA0125_H.A19790101.1300.020.nc'
nfin=Dataset(fnm,"r")
lons=nfin.variables["lon"][:]
lats=nfin.variables["lat"][:]
tair=nfin.variables["Tair"][:]
nfin.close()
print(np.shape(tair))
#determine if the point is within the domain
if lons[0]>lonc or lons[-1]<lonc or lats[0]>latc or lats[-1]<latc:
    print('site, lon=%f,lat=%f'%(lonc,latc))
    print('data, lon=%f,%f, lat=%f,%f'%(lons[0],lons[-1],lats[0],lats[-1]))
    raise Exception('The required site is not within the domain of climate forcing.')

#get coordinates of the requested site
dlon=lons[1]-lons[0]
dlat=lats[1]-lats[0]
fj1=(lonc-lons[0])/dlon
fj2=(latc-lats[0])/dlat
if fj1%1 <= 0.5:
    jlon=int(fj1)
else:
    jlon=int(fj1)+1

if fj2%1 <= 0.5:
    jlat=int(fj2)
else:
    jlat=int(fj2)+1
print('%d,%d'%(jlat,jlon))    
#variables
#time, hours since 1979-01-01.
#Tair, K, 2-meter above ground temperature. [T]K
#Qair, kg kg-1, 2-meter above ground specific humidity.     q [H], turn into mixing ratio w=q/(1-q), is height correction necessary?
#Psurf, Pa, Surface pressure. 
#Wind_E, m s-1, 10-meter above ground zonal wind speed.      [W]
#Wind_N, m s-1, 10-meter above ground meridional wind speed. [W]
#LWdown, W m-2, Surface incident longwave radiation          [L]
#Rainf, kg m-2 h-1, Total precipitation ~ mm h-1             [P]
#SWdown, W m-2, Shortwave radiation flux downwards (surface) [R]
#ecosim/ecosys requires Tair, Qair, Wind=sqrt(Wind_E^2+wind_N^2), LWdown, Rainf, and SWdown
#code KGSMW
site='US-Fuf'
vars=['Tair','Qair','Wind_E','Wind_N','Rainf','SWdown']
climcode='HJ0305XDHTHWPR'  #line 1
unitcode='KGSMW'           #line 2
z0iflgwznoon=[10.0,1.0,19.5]
for year in range(years[0],years[1]):
    wflnm=wordir+'weather/'+site+'w'+f'{year}'
    with open(wflnm,'w') as wfl:
        wfl.write(climcode+'\n')
        wfl.write(unitcode+'\n')

        isleap=(year%400 ==0) or (year%400!=0 and year%4==0)
        if isleap:
            leap=1
        else:
            leap=0
        z0iflgwznoon[2]=solar_noon(leap,180,lonc)
        wfl.write('%f,%f,%f\n'%(z0iflgwznoon[0],z0iflgwznoon[1],z0iflgwznoon[2]))
        wfl.write('7,0.25,0.25,0.0,0,0,0,0,0,0,0,0,,,,,,,\n')        
        jday=0
        for mm in range(0,12):
            das=daz[mm]        
            if mm==1 and isleap:
                das=das+1
            for dd in range(das):
                jday=jday+1
                for hh in range(24):
                    fnm=datadir+'NLDAS_FORA0125_H.A'+'%4d'%(year)+'%2.2d'%(mm+1)+'%2.2d'%(dd+1) \
                        +'.'+'%2.2d'%(hh)+'00.020.nc'
                    path = Path(fnm)
                    if path.is_file():
#                        print(fnm)
                        nfin=Dataset(fnm,"r")
                        datv=[]
                        for v in vars:
                            dat=nfin.variables[v][:]
                            datv.append(dat[0,jlat,jlon])
                    
                        datvv=[year,jday,hh,datv[0],datv[1],np.sqrt(datv[2]**2+datv[3]**2),\
                               datv[4],datv[5]]
    
                
                        wfl.write('%d,%d,%d,%f,%f,%f,%f,%f\n'%(datvv[0],datvv[1],datvv[2],datvv[3],\
                                                               datvv[4],datvv[5],datvv[6],datvv[7]))
                        nfin.close()